In [ ]:
from dotenv import load_dotenv

from IPython.display import Image, display
import gradio as gr

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_ollama import ChatOllama

from typing import Annotated, TypedDict
import getpass4
import os

In [ ]:
load_dotenv(override=True)

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass4.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langgraph.checkpoint.memory import MemorySaver

serper = GoogleSerperAPIWrapper()
# serper.run("What is the capital of France?")

memory = MemorySaver()

In [ ]:
from langchain.agents import Tool

tool_search = Tool(
    name="search",
    description="Useful for when you need more information from an online search",
    func=serper.run,
)

In [ ]:
# tool_search.invoke("What is the capital of France?")

In [ ]:
def calculate_math(expression: str) -> str:
    """Calculate mathematical expressions safely"""
    try:
        # Remove any potentially dangerous characters and evaluate safely
        allowed_chars: set[str] = set("0123456789+-*/(). ")
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"ผลลัพธ์: {expression} = {result}"
        else:
            return "ข้อผิดพลาด: นิพจน์ไม่ถูกต้อง ใช้ได้เฉพาะตัวเลขและเครื่องหมาย +, -, *, /, (, ) เท่านั้น"
    except Exception as e:
        return f"ข้อผิดพลาดในการคำนวณ: {str(e)}"


calculate_tool = Tool(
    name="calculator",
    func=calculate_math,
    description="Use this to calculate mathematical expressions. Input should be a valid math expression like '2 + 3 * 4' or '(10 + 5) / 3'",
)

calculate_tool.invoke("1+2")

#### Create Tools


In [ ]:
tools = [calculate_tool, tool_search]

#### Step 1: Define the State object


In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

#### Step 2: Start the Graph Builder with this State class


In [ ]:
graph_builder = StateGraph(State)

In [ ]:
llm = ChatOllama(model="gpt-oss")

llm_with_tools = llm.bind_tools(tools)

#### Step 3: Create a Node


In [ ]:
def chatbot(state: State) -> State:
    """This function is called when the user sends a message to the chatbot."""

    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

### Step 4: Create Edges


In [ ]:
graph_builder.add_conditional_edges("chatbot", tools_condition, {"tools": "tools"})
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

### Step 5: Compli the Graph


In [ ]:
graph = graph_builder.compile(checkpointer=memory)

display(Image(graph.get_graph().draw_mermaid_png()))

#### This's is? And, let's do this.


In [ ]:
config = {"configurable": {"thread_id": "123"}}

def chat(user_input: str, history) -> str:
    """This function is called when the user sends a message to the chatbot."""
    result = graph.invoke(
        {"messages": [{"role": "user", "content": user_input}]}, config=config
    )
    print(result)
    
    return result["messages"][-1].content



gr.ChatInterface(chat, type="messages").launch()

In [ ]:
graph.get_state(config)

In [ ]:
list(graph.get_state_history(config))

## OK it's time to add Memory!

### BUT WAIT!

We have this whole Graph maintaining the state and appending to the state.  
**Why isn't this handling memory?**

---

## This is a crucial point for understanding LangGraph

> A super-step can be considered a single iteration over the graph nodes.  
> Nodes that run in parallel are part of the same super-step,  
> while nodes that run sequentially belong to separate super-steps.

- One **"Super-Step"** of the graph represents one invocation of passing messages between agents.
- In idiomatic LangGraph, you call `invoke()` to run your graph for each super-step; for each interaction.
- The reducer handles state updates automatically **within one super-step**, but **not between them**.

That is what **checkpointing** achieves.


In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory: MemorySaver = MemorySaver()

In [ ]:
graph_builder = StateGraph(State)

llm = ChatOllama(model="llama3.1")

llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State) -> State:
    """This function is called when the user sends a message to the chatbot."""

    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

graph_builder.add_conditional_edges("chatbot", tools_condition, {"tools": "tools"})
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
# graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(checkpointer=memory)

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "222"}}

def chat(user_input: str, history) -> str:
    """This function is called when the user sends a message to the chatbot."""
    result = graph.invoke(
        {"messages": [{"role": "user", "content": user_input}]}, config=config
    )
    print(result)
    
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

### LangChain gives you tools to set the state back to a prior point in time, to branch off:

```
config = {"configurable": {"thread_id": "1", "checkpoint_id": ...}}
graph.invoke(None, config=config)
```

And this allows you to build stable systems that can be recovered and rerun from any prior checkpoint.

And now let's store in SQL

And this is the power of LangGraph.


In [ ]:
import sqlite3
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver


db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)

# Step 3
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
# graph_builder.add_conditional_edges("chatbot", tools_condition, {"tools": "tools"})
graph_builder.add_conditional_edges(
    "chatbot", tools_condition, {"tools": "tools", "__end__": END}
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# graph_builder.add_conditional_edges("chatbot", tools_condition, {"tools": "tools"})
# graph_builder.add_edge("tools", "chatbot")
# graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "3333"}}


def chat(user_input: str, history) -> str:
    """This function is called when the user sends a message to the chatbot."""
    result = graph.invoke(
        {"messages": [{"role": "user", "content": user_input}]}, config=config
    )
    print(result)
    
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()